# Compare Temperature and Salinity from ACCESS-OM2 to WOA13

This notebook shows examples of comparing ACCESS-OM2 Temperature and Salinity structure to the WOA13 climatology (that is used as initial conditions for most runs). We describe the location and setup of the WOA13 data interpolated onto the model grids, as well as plot SST and SSS anomalies along with equatorial slices of temperature and salinity anomalies.

First, lets load in some modules, call some workers and load a database

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np
import cftime
import datetime
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import sys, os, warnings
warnings.filterwarnings('ignore')
import intake
catalog = intake.cat.access_nri
from dask.distributed import Client

In [2]:
client = Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41833,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42871,Total threads: 1
Dashboard: /proxy/38665/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:39055,


## WOA13 data

The WOA13 data has already been interpolated onto the various model grids (as it is used for initial conditions). This makes it easy to plot biases.

The WOA13 data is located in the `/g/data/ik11/observations/woa13/` folder, with various subfolders for the different resolutions (including the different vertical grids such as KDS50, KDS75 etc.). The available interpolated versions are (see `/g/data/ik11/observations/woa13/README/`:

- `woa13/10` - 1-degree, GFDL50 vertical levels scheme
- `woa13/025` - 1/4-degree, GFDL50 vertical levels scheme
- `woa13/01` - 1/10-degree, KDS75 vertical levels scheme
- `woa13/10_KDS50` - 1-degree, KDS50 vertical levels scheme
- `woa13/025_KDS50` - 1/4-degree, KDS50 vertical levels scheme

Note that the new ACCESS-OM2 runs all use the KDS vertical levels schemes (KDS50 at 1-degree and 1/4-degree, KDS75 at 1/10-degree).

Let's first explore some of this data by looking at the 1-degree KDS50 experiment. First the netcdf files. Note that currently, because of the folder structure of the WOA13 data, the experiment names can be a bit opaque as they do not contain the woa13 string. Here we examine the 1-degree KDS50 data

In [3]:
cat_subset = catalog.search(name="WOA-13")

In [4]:
woa13_10 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/10/.*mom10.nc").to_dask(xarray_open_kwargs={"use_cftime": True})
woa13_025 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/025/.*mom025.nc").to_dask(xarray_open_kwargs={"use_cftime": True})
woa13_01 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/01/.*mom01.nc").to_dask(xarray_open_kwargs={"use_cftime": True})
woa13_10_KDS50 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/10_KDS50/.*mom10.nc").to_dask(xarray_open_kwargs={"use_cftime": True})
woa13_025_KDS50 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/025_KDS50/.*mom025.nc").to_dask(xarray_open_kwargs={"use_cftime": True})
woa13_10_KDS75 = cat_subset["WOA-13"].search(path="/g/data/ik11/observations/woa13/10_KDS75/.*mom10.nc").to_dask(xarray_open_kwargs={"use_cftime": True})

## Comparing to ACCESS-OM2 simulations

Now lets plot some biases against the WOA13 data set. We will use the original ACCESS-OM2 IAF runs, so first define a dictionary with information on those runs

In [5]:
from collections import OrderedDict
exptdict = OrderedDict([
    ('1degIAF', # 1deg IAF run from Kiss et al. 2020
     {'model': 'ACCESS-OM2 IAF', 'expt': '1deg_jra55v13_iaf_spinup1_B1',
      'n_files': -12, 'itime': '1998-01-01', 'ftime': None}),
    ('025degIAF', # 025deg IAF run from Kiss et al. 2020
     {'model': 'ACCESS-OM2-025 IAF', 'expt': '025deg_jra55v13_iaf_gmredi6',
      'n_files': -34, 'itime': '1998-01-01', 'ftime': None}),
    ('01degIAF', # 01deg IAF run from Kiss et al. 2020
     {'model': 'ACCESS-OM2-01 IAF',  'expt': '01deg_jra55v13_iaf',
      'n_files': None, 'itime': '1998-01-01','ftime': None})
])

For each of these runs we then attach information to these dictionaries containing the matched WOA13 interpolated data sets. Note that we include a file name (with wildcards) so that we only use the monthly files and not the additional `ocean_temp_salt.res.nc` initial condition file.

In [6]:
# Add on pre-interpolated WOA13 directories for every run:
for ekey in exptdict.keys():
    e = exptdict[ekey]
    if (ekey.find('025deg') != -1):
        e['WOA13expt'] = '025_KDS50'
        e['WOA13array'] = woa13_025_KDS50
    elif (ekey.find('01deg') != -1):
        e['WOA13expt'] = '01'
        e['WOA13array'] = woa13_01
    else:
        e['WOA13expt'] = '10_KDS50'
        e['WOA13array'] = woa13_10_KDS50

## SST and SSS biases

We will start by plotting SST and SSS biases compared to WOA13. The following loop loads data from the model runs and the corresponding WOA13 data and saves them into the previous dictionary (as entries SST, SST_WOA13 and SST_anom and the same for SSS). This can take time...

In [33]:
# Function to extract and load SST and SSS from the models, WOA13
for ekey in exptdict.keys():
    e = exptdict[ekey]
    
    # SST
    
    # Load surface temperature from model
    cat_subset = catalog[e['expt']]
    var_search = cat_subset.search(variable='temp')
    darray = var_search.to_dask()
    darray = darray['temp']
    darray = darray.sel(time=slice(e['itime'], e['ftime']))
    surface_temp = darray.isel(st_ocean=0)

    # convert MOM time to datetime:
    tstart = datetime.datetime.fromtimestamp(surface_temp.time.item(0) * 1e-9)
    tend = datetime.datetime.fromtimestamp(surface_temp.time.item(-1) * 1e-9)
    
    # Extract a year range string and print (for title string):
    e['yearrange'] = "{} to {}".format(tstart.strftime("%Y-%m"), tend.strftime("%Y-%m"),)
    print(f"{ekey}: {e['yearrange']}")
    
    # Add SST to dictionary
    e['SST'] = surface_temp.mean('time').load() - 273.15
    
    # Load WOA13 SST and add to dictionary
    darray = e['WOA13array']
    e['SST_WOA13'] = darray.isel(ZT=0).mean('time').load()
    
    # Calculate bias and add to dictionary
    SST_anom = e['SST'] - e['SST_WOA13'].values 
    e['SST_anom'] = SST_anom.load()
    
    # SSS
    cat_subset = catalog[e['expt']]
    var_search = cat_subset.search(variable='salt')
    darray = var_search.to_dask()
    darray = darray['salt']
    darray = darray.sel(time=slice(e['itime'], e['ftime']))
    surface_salt = darray.isel(st_ocean=0)
    e['SSS'] = surface_salt.mean('time').load()
    
    # Load WOA13 SSS and add to dictionary
    darray = e['WOA13array']
    darray = darray['salt']
    e['SSS_WOA13'] = darray.isel(ZT=0).mean('time').load()

    SSS_anom = e['SSS'] - e['SSS_WOA13'].values 
    e['SSS_anom'] = SSS_anom.load()

KeyError: "key='025deg_jra55v13_iaf_gmredi6' not found in catalog. You can access the list of valid source keys via the .keys() method."

Now that all the data is loaded, all we have to do is plot it.

We first define a function to plot the SST

In [ ]:
def plot_SST(ekeys):
    clev = np.arange(-3, 3.25, 0.25)
    land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                       edgecolor='black',
                                       facecolor='gray', linewidth=0.5)

    for i, ekey in enumerate(ekeys):
        e = exptdict[ekey]
        ax1 = plt.subplot(1+len(ekeys)//2, 2, i+1,
                          projection=ccrs.Robinson(central_longitude=-100))
        ax1.coastlines(resolution='50m')
        ax1.add_feature(land_50m)
        pn = e['SST_anom'].plot.contourf(cmap=cm.cm.balance, levels=clev,
                                        add_colorbar=False, transform=ccrs.PlateCarree())
        plt.title("({}) {}, {}".format(chr(ord('a') + i), e['model'], e['yearrange']))

        if i == 0:#1:
            # save plot for colourbar
            p0 = pn

    i = i+1
    e = exptdict['01degIAF']
    ax1 = plt.subplot(1+len(ekeys)//2, 2, i+1,
                      projection=ccrs.Robinson(central_longitude=-100))
    ax1.coastlines(resolution='50m')
    ax1.add_feature(land_50m)
    pn = e['SST_WOA13'].plot.contourf(cmap=cm.cm.thermal, levels=np.arange(-2., 32., 1.),
                                      add_colorbar=False, transform=ccrs.PlateCarree())
    plt.title("({}) WOA13".format(chr(ord('a') + i)))

    ax5 = plt.axes([0.92, 0.52, 0.01, 0.33])
    cb = plt.colorbar(p0, cax=ax5, orientation='vertical')
    cb.ax.set_ylabel('SST anomaly (°C)')

    ax6 = plt.axes([0.92,0.13,0.01,0.33])
    cb = plt.colorbar(pn, cax=ax6, orientation='vertical')
    cb.ax.set_ylabel('SST (°C)')

We now plot IAF and RYF SST biases at 3 resolutions:

In [ ]:
fig = plt.figure(figsize=(14, 10))
ekeys = ['01degIAF', '025degIAF', '1degIAF']
plot_SST(ekeys)

Then we do the same for sea surface salinity biases

In [ ]:
def plot_SSS(ekeys):
    clev = np.arange(-1.5, 1.6, 0.1)
    land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                       edgecolor='black',
                                       facecolor='gray',linewidth=0.5)

    for i, ekey in enumerate(ekeys):
        e = exptdict[ekey]
        ax1 = plt.subplot(1+len(ekeys)//2, 2, i+1,
                          projection=ccrs.Robinson(central_longitude=-100))
        ax1.coastlines(resolution='50m')
        ax1.add_feature(land_50m)
        pn = e['SSS_anom'].plot.contourf(cmap=cm.cm.balance, levels=clev,
                                         add_colorbar=False, transform=ccrs.PlateCarree())
        plt.title("({}) {}, {}".format(chr(ord('a') + i), e['model'], e['yearrange']))
        
        if i == 1:
            # save plot for colourbar
            p0 = pn

    i = i+1
    e = exptdict['01degIAF']
    ax1 = plt.subplot(1+len(ekeys)//2, 2, i+1,
                      projection=ccrs.Robinson(central_longitude=-100))
    ax1.coastlines(resolution='50m')
    ax1.add_feature(land_50m)
    pn = e['SSS_WOA13'].plot.contourf(cmap=cm.cm.thermal, levels=np.arange(31., 36.2, 0.2),
                                      add_colorbar=False, transform=ccrs.PlateCarree())
    plt.title("({}) WOA13".format(chr(ord('a') + i)))

    ax5 = plt.axes([0.92, 0.52, 0.01, 0.33])
    cb = plt.colorbar(p0, cax=ax5, orientation='vertical')
    cb.ax.set_ylabel('SSS anomaly (psu)')

    ax6 = plt.axes([0.92, 0.13, 0.01, 0.33])
    cb = plt.colorbar(pn, cax=ax6, orientation='vertical')
    cb.ax.set_ylabel('SSS (psu)')

Plot IAF and RYF SSS biases at 3 resolutions:

In [ ]:
fig = plt.figure(figsize=(14, 10))
ekeys = ['01degIAF', '025degIAF', '1degIAF']
plot_SSS(ekeys)

## Equatorial Pacific Temperature and Salinity Longitude-depth biases

Our final example compares temperature and salinity biases in the tropical Pacific (note, this overlaps somewhat with the `Equatorial_thermal_and_zonal_velocity_structure.ipynb` documented example).

We follow the same procedure as before, first loading the data.

In [ ]:
# Define list of experiments to load (useful for testing):
ekeys = ['1degIAF', '025degIAF', '01degIAF']

# Loop through models
for ekey in ekeys:
    e = exptdict[ekey]
    
    # Load temperature
    cat_subset = catalog[e['expt']]
    var_search = cat_subset.search(variable='temp')
    darray = var_search.to_dask()
    darray = darray['temp']
    darray = darray.sel(time=slice(e['itime'], e['ftime']))
    eq_temp = darray.sel(yt_ocean=0, method='nearest')

    # convert MOM time to datetime:
    tstart = datetime.datetime.fromtimestamp(eq_temp.time.item(0) * 1e-9)
    tend = datetime.datetime.fromtimestamp(eq_temp.time.item(-1) * 1e-9)
    
    # Set a text string to add the year range in title.
    e['yearrange'] = "{} to {}".format(tstart.strftime("%Y-%m"), tend.strftime("%Y-%m"),)
    print(f"{ekey}: {e['yearrange']}")

    # Extract the WOA13 data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        cat_subset = catalog[e['WOA13expt']]
        var_search = cat_subset.search(variable='temp')
        darray = var_search.to_dask()
        darray = darray['temp']
        e['eq_temp_WOA13'] = darray.sel(GRID_Y_T=0., method='nearest').mean('time')
    
    # Calculate the bias
    eq_temp_anom = eq_temp.mean('time') - 273.15 - e['eq_temp_WOA13'].values
    eq_temp_anom.attrs['units'] = 'degrees Celsius'
    
    e['eq_temp_anom'] = eq_temp_anom.load()
    
    # Do salt as for temperature above
    cat_subset = catalog[e['expt']]
    var_search = cat_subset.search(variable='salt')
    darray = var_search.to_dask()
    darray = darray['salt']
    darray = darray.sel(time=slice(e['itime'], e['ftime']))
    eq_salt = darray.sel(yt_ocean=0, method='nearest')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        cat_subset = catalog[e['WOA13expt']]
        var_search = cat_subset.search(variable='salt')
        darray = var_search.to_dask()
        darray = darray['salt']
        e['eq_salt_WOA13'] = darray.sel(GRID_Y_T=0.,method='nearest').mean('time')
    eq_salt_anom = eq_salt.mean('time') - e['eq_salt_WOA13'].values
    e['eq_salt_anom'] = eq_salt_anom.load()

Then plotting equatorial plots of temperature biases

In [ ]:
# Define a function to plot Equatorial Slices of temperature:
def plot_eqtemp(ekeys):
    
    # Define contour levels
    clev = np.arange(-3.,3.25,0.25)

    # Loop through models
    for i, ekey in enumerate(ekeys):
        e = exptdict[ekey]
        ax1 = plt.subplot(int(np.ceil(len(ekeys)/2)), 2, i+1)
        
        # Plot bias as color
        pn = e['eq_temp_anom'].plot.contourf(cmap='bwr', levels=clev, add_colorbar=False, yincrease=False)
        
        # Plot WOA13 isotherms (and 20C bold)
        CS = e['eq_temp_WOA13'].plot.contour(levels=np.arange(0, 32, 2), colors='k')
        ax1.clabel(CS, inline=False, fmt='%d', fontsize=15)
        e['eq_temp_WOA13'].plot.contour(levels=[20.], colors='k', linewidths=3.)
        (e['eq_temp_anom'] + e['eq_temp_WOA13'].values).plot.contour(levels=[20.], colors='k',
                                                                     linewidths=3., linestyles='--')
        
        # Add annotations
        plt.title("({}) {}, {}".format(chr(ord('a') + i), e['model'], e['yearrange']))
        ax1.set_ylim([300., 0.])
        ax1.set_xlim([-220., -80.])
        ax1.set_ylabel('Depth (m)')
        ax1.set_xlabel('Longitude ($^\circ$E)')
        
        if i == 0:
            ax1.text(-210., 275., 'WOA13 Isotherms', fontsize=15)
            p0 = pn

    ax5 = plt.axes([0.92, 0.2, 0.01, 0.5])
    cb = plt.colorbar(p0, cax=ax5, orientation='vertical')
    cb.ax.set_ylabel('Temperature anomaly (°C)')

Plot temperature comparison of IAF simulations.

In [ ]:
fig = plt.figure(figsize=(14, 12))
ekeys = ['1degIAF', '025degIAF', '01degIAF']
plot_eqtemp(ekeys)

And finally, repeat the same for salinity.

In [ ]:
# Define a function to plot Equatorial Slices of salinity
def plot_eqsalt(ekeys):
    
    # Define contour levels
    clev = np.arange(-1., 1.1, 0.1)

    # Loop through models
    for i, ekey in enumerate(ekeys):
        e = exptdict[ekey]
        ax1 = plt.subplot(int(np.ceil(len(ekeys)/2)), 2, i+1)
        
        # Plot bias as color
        pn = e['eq_salt_anom'].plot.contourf(cmap='bwr', levels=clev, add_colorbar=False, yincrease=False)
        
        # Plot WOA13 salinity (and 20C bold)
        CS = e['eq_salt_WOA13'].plot.contour(levels=np.arange(30., 36.1, 0.1), colors='k')
        ax1.clabel(CS, inline=False, fmt='%3.2f', fontsize=15)
        
        # Add annotations
        plt.title("({}) {}, {}".format(chr(ord('a') + i), e['model'], e['yearrange']))
        ax1.set_ylim([300., 0.])
        ax1.set_xlim([-220., -80.])
        ax1.set_ylabel('Depth (m)')
        ax1.set_xlabel('Longitude ($^\circ$E)')
        
        if i == 0:
            ax1.text(-210., 275., 'WOA13 Isohalines', fontsize=15)
            p0 = pn

    ax5 = plt.axes([0.92, 0.2, 0.01, 0.5])
    cb = plt.colorbar(p0, cax=ax5, orientation='vertical')
    cb.ax.set_ylabel('Salinity anomaly (psu)')

Plot salinity comparisson for IAF simulations.

In [ ]:
fig = plt.figure(figsize=(14, 12))
ekeys = ['1degIAF', '025degIAF', '01degIAF']
plot_eqsalt(ekeys)